In [2]:
import pandas as pd
import json
import re
import os
import requests
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
passivator_df = pd.read_csv("data/final_extraction/df_has_passivator.csv")
passivator_df

Unnamed: 0                     perovskite_composition  \
0              0                                     MAPbI3   
1              1                                        NaN   
2              3               Cs 0.05 FA 0.9 MA 0.05 PbI 3   
3              4  Cs 0.17 FA 0.83 Pb(I 0.9 Br 0.1 ) 3 (WBg)   
4              5                FAPbI3 0.85 (-MAPbBr3) 0.15   
...          ...                                        ...   
2001        2532                                     MAPbI3   
2002        2533                                     MAPbI3   
2003        2534                     Cu2ZnSn(S,Se)4 (C:SSe)   
2004        2536                                        NaN   
2005        2538                                        NaN   

     electron_transport_layer pin_nip_structure hole_transport_layer  \
0                   PEDOT:PSS               NIP                 PCBM   
1                         NaN               NaN                  NaN   
2                         NaN               NaN                  NaN   
3                        TiO2               NaN         Spiro-OMeTAD   
4                Spiro-OMeTAD               NIP                  NaN   
...                       ...               ...                  ...   
2001                      NaN               NaN                  NaN   
2002                     PCBM               PIN                 NiOx   
2003                      NaN               NaN                  NaN   
2004                     TiO2               NaN         Spiro-MeOTAD   
2005                      NaN               NaN                  NaN   

                          passivating_molecule  control_pce  control_voc  \
0                                         DAAB         4.50        0.870   
1              formamidine sulfinic acid (FSA)          NaN          NaN   
2                      Cyanoguanidine diiodide        20.44     1077.000   
3                                        tBBAI        14.91        1.070   
4     7-amino-4-trifluoromethylcoumarin (C151)        20.44        1.090   
...                                        ...          ...          ...   
2001                                      OTTM        18.45        1.080   
2002                                       PbS        15.26        1.002   
2003                              Vanadium (V)         8.28        0.454   
2004                                     K-NAA        19.86        1.140   
2005                                    IMBF 4          NaN          NaN   

      treated_pce  treated_voc  ... treated_eqe treated_stabilized_pce  \
0           12.60        0.920  ...         NaN                    NaN   
1           27.00          NaN  ...         NaN                    NaN   
2           23.04     1119.000  ...         NaN                    NaN   
3           23.50        1.140  ...         NaN                    NaN   
4           22.46        1.140  ...         NaN                    NaN   
...           ...          ...  ...         ...                    ...   
2001        21.01        1.120  ...         NaN                    NaN   
2002        18.74        1.025  ...         NaN                    NaN   
2003        11.11        0.458  ...         NaN                    NaN   
2004          NaN          NaN  ...         NaN                    NaN   
2005        23.05        1.150  ...         NaN                    NaN   

     control_fill_factor  treated_fill_factor  controljsc treatedjsc  \
0                    NaN                  NaN         NaN        NaN   
1                    NaN                  NaN         NaN        NaN   
2                    NaN                  NaN         NaN        NaN   
3                    NaN                  NaN         NaN        NaN   
4                    NaN                  NaN         NaN        NaN   
...                  ...                  ...         ...        ...   
2001                 NaN                  NaN         NaN        NaN   
2002  

In [52]:
passivator_conversions_path = "data/passivator_conversions.json"
with open(passivator_conversions_path, "r", encoding="utf-8") as f:
    passivator_conversions = json.load(f)

In [53]:
def get_IUPAC_name(passivator):
    if passivator in passivator_conversions:
        return passivator_conversions[passivator]
    else:
        return "unknown"

In [54]:
passivator_df["IUPAC_passivator"] = passivator_df["passivating_molecule"].apply(get_IUPAC_name)
passivator_df

Unnamed: 0                     perovskite_composition  \
0              0                                     MAPbI3   
1              1                                        NaN   
2              3               Cs 0.05 FA 0.9 MA 0.05 PbI 3   
3              4  Cs 0.17 FA 0.83 Pb(I 0.9 Br 0.1 ) 3 (WBg)   
4              5                FAPbI3 0.85 (-MAPbBr3) 0.15   
...          ...                                        ...   
2001        2532                                     MAPbI3   
2002        2533                                     MAPbI3   
2003        2534                     Cu2ZnSn(S,Se)4 (C:SSe)   
2004        2536                                        NaN   
2005        2538                                        NaN   

     electron_transport_layer pin_nip_structure hole_transport_layer  \
0                   PEDOT:PSS               NIP                 PCBM   
1                         NaN               NaN                  NaN   
2                         NaN               NaN                  NaN   
3                        TiO2               NaN         Spiro-OMeTAD   
4                Spiro-OMeTAD               NIP                  NaN   
...                       ...               ...                  ...   
2001                      NaN               NaN                  NaN   
2002                     PCBM               PIN                 NiOx   
2003                      NaN               NaN                  NaN   
2004                     TiO2               NaN         Spiro-MeOTAD   
2005                      NaN               NaN                  NaN   

                          passivating_molecule  control_pce  control_voc  \
0                                         DAAB         4.50        0.870   
1              formamidine sulfinic acid (FSA)          NaN          NaN   
2                      Cyanoguanidine diiodide        20.44     1077.000   
3                                        tBBAI        14.91        1.070   
4     7-amino-4-trifluoromethylcoumarin (C151)        20.44        1.090   
...                                        ...          ...          ...   
2001                                      OTTM        18.45        1.080   
2002                                       PbS        15.26        1.002   
2003                              Vanadium (V)         8.28        0.454   
2004                                     K-NAA        19.86        1.140   
2005                                    IMBF 4          NaN          NaN   

      treated_pce  treated_voc  ... treated_stabilized_pce  \
0           12.60        0.920  ...                    NaN   
1           27.00          NaN  ...                    NaN   
2           23.04     1119.000  ...                    NaN   
3           23.50        1.140  ...                    NaN   
4           22.46        1.140  ...                    NaN   
...           ...          ...  ...                    ...   
2001        21.01        1.120  ...                    NaN   
2002        18.74        1.025  ...                    NaN   
2003        11.11        0.458  ...                    NaN   
2004          NaN          NaN  ...                    NaN   
2005        23.05        1.150  ...                    NaN   

     control_fill_factor treated_fill_factor  controljsc  treatedjsc  \
0                    NaN                 NaN         NaN         NaN   
1                    NaN                 NaN         NaN         NaN   
2                    NaN                 NaN         NaN         NaN   
3                    NaN                 NaN         NaN         NaN   
4                    NaN                 NaN         NaN         NaN   
...                  ...                 ...         ...         ...   
2001                 NaN                 NaN         NaN         NaN   
2002                 NaN                 NaN         NaN         NaN   
2003                 NaN                 NaN         NaN         NaN   
2004  

In [ ]:
null_passivators = passivator_df[passivator_df["IUPAC_passivator"].isna()]
passivators_list = null_passivators["passivating_molecule"].tolist()

Unnamed: 0                     perovskite_composition  \
1              1                                        NaN   
3              4  Cs 0.17 FA 0.83 Pb(I 0.9 Br 0.1 ) 3 (WBg)   
4              5                FAPbI3 0.85 (-MAPbBr3) 0.15   
7              8                                        NaN   
8              9                                     FAPbI3   
...          ...                                        ...   
1994        2522     FA 0.7 MA 0.2 Cs 0.1 Sn 0.5 Pb 0.5 I 3   
2000        2530                                        NaN   
2001        2532                                     MAPbI3   
2004        2536                                        NaN   
2005        2538                                        NaN   

     electron_transport_layer pin_nip_structure hole_transport_layer  \
1                         NaN               NaN                  NaN   
3                        TiO2               NaN         Spiro-OMeTAD   
4                Spiro-OMeTAD               NIP                  NaN   
7                         NaN               NaN                  NaN   
8                        SnO2               NIP         Spiro-MeOTAD   
...                       ...               ...                  ...   
1994                     PCBM               PIN            PEDOT:PSS   
2000                      NaN               NaN                  NaN   
2001                      NaN               NaN                  NaN   
2004                     TiO2               NaN         Spiro-MeOTAD   
2005                      NaN               NaN                  NaN   

                          passivating_molecule  control_pce  control_voc  \
1              formamidine sulfinic acid (FSA)          NaN          NaN   
3                                        tBBAI        14.91        1.070   
4     7-amino-4-trifluoromethylcoumarin (C151)        20.44        1.090   
7                                         ICBA        22.40        1.150   
8                              MASCN and FASCN        25.20        1.255   
...                                        ...          ...          ...   
1994              𝛽-poly(1,1-difluoroethylene)          NaN          NaN   
2000                                    TDMASn        15.00          NaN   
2001                                      OTTM        18.45        1.080   
2004                                     K-NAA        19.86        1.140   
2005                                    IMBF 4          NaN          NaN   

      treated_pce  treated_voc  ... treated_stabilized_pce  \
1           27.00          NaN  ...                    NaN   
3           23.50         1.14  ...                    NaN   
4           22.46         1.14  ...                    NaN   
7           25.10         1.19  ...                    NaN   
8           22.40         1.19  ...                    NaN   
...           ...          ...  ...                    ...   
1994        23.44         0.88  ...                    NaN   
2000        18.20          NaN  ...                    NaN   
2001        21.01         1.12  ...                    NaN   
2004          NaN          NaN  ...                    NaN   
2005        23.05         1.15  ...                    NaN   

     control_fill_factor treated_fill_factor  controljsc  treatedjsc  \
1                    NaN                 NaN         NaN         NaN   
3                    NaN                 NaN         NaN         NaN   
4                    NaN                 NaN         NaN         NaN   
7                    NaN                 NaN         NaN         NaN   
8                    NaN                 NaN         NaN         NaN   
...                  ...                 ...         ...         ...   
1994                 NaN                 NaN         NaN         NaN   
2000                 NaN                 NaN         NaN         NaN   
2001                 NaN                 NaN         NaN         NaN   
2004  

In [30]:
def get_unconverted_passivators(passivators_list):
    passivators = set()
    for item in passivators_list:
        passivators.add(item)
    passivators = [passivator for passivator in passivators if passivator not in passivator_conversions and passivator not in [None, 'none']]
    return passivators    
        

In [31]:
PREFIX = """
Role:
You are a helpful scientific assistant specializing in passivating molecules for perovskite solar cells.

Task:
Given a list of molecule names, provide the full name of each molecule in IUPAC nomenclature. The IUPAC name must be parseable into SMILES format.

Rules:
-If the molecule name is already in IUPAC format, return it as is.
-If there are multiple molecule names given, just use the first one.
-If the name contains additional descriptive words (e.g., "passivating," "functionalized"), extract only the molecule name and convert it to IUPAC format.
-If the full molecule name cannot be parsed into SMILES after fully reasoning through it multiple times, return null.
-Ensure no molecules are left out.

Output Format:
Provide a JSON object where each key is the provided molecule name and the value is the corresponding IUPAC name or null.

json
Copy
{
    "molecule_name_1": "IUPAC_name_or_null",
    "molecule_name_2": "IUPAC_name_or_null"
}
Important Notes:

-Focus only on the parts of the string that represent the molecule name.

-Double-check that the IUPAC name can be parsed into SMILES. If not, return null.
-If the molecule is not relevant to passivating perovskite solar cells, return null.

Example Input:

{
    "ethylammonium bromide": "ethylammonium bromide",
    "passivating 2-phenylethylamine": "2-phenylethylamine",
    "CF3PEAI": "2-(4-(Trifluoromethyl)phenyl)ethylammonium iodide",
    "2D perovskite": null
}
Begin converting!
"""

In [33]:
def get_new_passivator_conversions(passivators, api_key_name="OPENAI_API_KEY", base_url="https://api.openai.com/v1/", model_name="gpt-4o"):
    api_key = os.getenv(api_key_name)
    client = OpenAI(api_key=api_key, base_url=base_url)
    system_content = PREFIX
    i = 0
    user_content = str(passivators)

    messages = [
        {
            "role": "system",
            "content": system_content
        },
        {
            "role": "user",
            "content": user_content
        }
    ]
    response = client.chat.completions.create(
        model=model_name,
        messages=messages,
        stream=False
    )
    output = response.choices[0].message.content
    json_match = re.search(r"\{.*\}", output, re.DOTALL)

    if json_match:
        raw_json = json_match.group(0).strip()
    else:
        print("No JSON found")
        return {}
    try:
        parsed_data = json.loads(raw_json)
        return parsed_data
    except json.JSONDecodeError as e:
        print("Error creating JSON", e)
        return {}

In [34]:
passivators_list = passivator_df["passivating_molecule"].tolist()

In [36]:
len(passivators_list)

2006

In [37]:
unconverted_passivators = get_unconverted_passivators(passivators_list)

In [39]:
len(unconverted_passivators)

329

In [40]:
new_passivators = get_new_passivator_conversions(unconverted_passivators)

In [41]:
new_passivators

{'2,5-di-tert-butylhydroquinone (DBHQ)': '2,5-di-tert-butylbenzene-1,4-diol',
 'Benzophenone (BP-5 and BP-9)': 'Diphenylmethanone',
 'Tanshinone IIA': 'null',
 '1-ethyl-3-methylimidazoliumiodide': '1-ethyl-3-methylimidazolium iodide',
 '1,3-dimethyl-3,4,5,6-tetrahydro-2(1H)-pyrimidinone (DMPU)': '1,3-dimethyl-2-imidazolidinone',
 'CoBr₂': 'Cobalt(II) bromide',
 'FeCO3, Fe2O3, Fe3O4, α-FeOOH, γ-FeOOH': 'null',
 'Imidazoanthraquinone derivative (AQ)': 'null',
 "4,4'-oxybisbenzoic acid (OBBA)": "4,4'-oxybisbenzoic acid",
 '[EMIM] + [PF6]^-': '1-ethyl-3-methylimidazolium hexafluorophosphate',
 "N, N'-Dimethyl-1,2-ethanediamine (DMEDA)": "N,N'-dimethylethane-1,2-diamine",
 "N,N'-di(4-pyridyl)-1,4,5,8-naphthalenetetracarboxdiimide (DPNDI)": "N,N'-bis(4-pyridyl)-1,4,5,8-naphthalene tetracarboxdiimide",
 'Benzylamine, phenylethylamine bromide, guanidinium bromide': 'null',
 'Europium (Eu^3+/Eu^2+) and sulfonate anthraquinone-based molecules, and thiol/disulfide couples': 'null',
 'OAI (octylam

In [42]:
new_passivators["PEATFA"]

'null'

In [43]:
### second pass to ensure not missing anything (GPT-4o takes less time for this)
null_passivators = [k for k, v in new_passivators.items() if v == None or v == "null"]
extracted_passivators = {k: v for k, v in new_passivators.items() if v != None or v != "null"}

second_pass = get_new_passivator_conversions(null_passivators)

In [44]:
extracted_passivators.update(second_pass)

In [45]:
### cleaning any multiples:
for key in extracted_passivators:
    value = extracted_passivators[key]
    if isinstance(value, dict):
        extracted_passivators[key] = value[list(value.keys())[0]]

In [46]:
extracted_passivators

{'2,5-di-tert-butylhydroquinone (DBHQ)': '2,5-di-tert-butylbenzene-1,4-diol',
 'Benzophenone (BP-5 and BP-9)': 'Diphenylmethanone',
 'Tanshinone IIA': None,
 '1-ethyl-3-methylimidazoliumiodide': '1-ethyl-3-methylimidazolium iodide',
 '1,3-dimethyl-3,4,5,6-tetrahydro-2(1H)-pyrimidinone (DMPU)': '1,3-dimethyl-2-imidazolidinone',
 'CoBr₂': 'Cobalt(II) bromide',
 'FeCO3, Fe2O3, Fe3O4, α-FeOOH, γ-FeOOH': None,
 'Imidazoanthraquinone derivative (AQ)': None,
 "4,4'-oxybisbenzoic acid (OBBA)": "4,4'-oxybisbenzoic acid",
 '[EMIM] + [PF6]^-': '1-ethyl-3-methylimidazolium hexafluorophosphate',
 "N, N'-Dimethyl-1,2-ethanediamine (DMEDA)": "N,N'-dimethylethane-1,2-diamine",
 "N,N'-di(4-pyridyl)-1,4,5,8-naphthalenetetracarboxdiimide (DPNDI)": "N,N'-bis(4-pyridyl)-1,4,5,8-naphthalene tetracarboxdiimide",
 'Benzylamine, phenylethylamine bromide, guanidinium bromide': None,
 'Europium (Eu^3+/Eu^2+) and sulfonate anthraquinone-based molecules, and thiol/disulfide couples': None,
 'OAI (octylammonium iod

In [47]:
passivator_conversions.update(extracted_passivators)

In [48]:
passivator_conversions

{'natural alkene lycopene': '2-(6,6,6-Trimethyl-1-cyclohexen-1-yl)-bicyclo[4.2.0]octa-1,3,5-triene',
 'F-15': None,
 '4-ethenyl-2,6-dimethoxyphenol (canolol, CNL)': '4-ethenyl-2,6-dimethoxyphenol',
 'PbS quantum dots': None,
 '1-methyl-3-propylimidazolium iodide (1-MPII)': '1-methyl-3-propylimidazolium iodide',
 'Cu2(Tu)Br2': None,
 'Gly-E': None,
 'MPA-TB-CA': None,
 'K-NAA': None,
 '4-Acetamidobenzoic acid': '4-acetamidobenzoic acid',
 'MoO_x': None,
 'morpholinium iodide (MPI)': 'morpholinium iodide',
 '2D perovskite nanoplatelets-PMMA composite': None,
 'N,N-methylenebisacrylamide (MBA)': 'N,N-methylenebisacrylamide',
 '4-methanesulfonyl-benzamidine hydrochloride (MSBH)': '4-methanesulfonyl-benzamidine hydrochloride',
 '3-(2-aminoethyl)pyridine (3-PyEA)': '3-(2-aminoethyl)pyridine',
 'MgO': 'magnesium oxide',
 'Nylon 11 (N11)': None,
 'GQD-doped ZnO': None,
 'PMMA': 'poly(methyl methacrylate)',
 'usnic acid': 'usnic acid',
 'AMSA': None,
 'TPCA': None,
 'FAMAPbI-BrCl': None,
 'Phen

In [49]:
with open('data/passivator_conversions.json', 'w') as f:
    json.dump(passivator_conversions, f)

In [50]:
results = {}
for key in passivator_conversions:
    passivator = passivator_conversions[key]
    if passivator is not None:
        print(passivator)
        base_url = "https://opsin.ch.cam.ac.uk/opsin/"
        smiles_url = base_url + passivator + ".smi"
        r = requests.get(smiles_url)
        result = r.text if r.status_code == 200 else None
        results[passivator] = result

2-(6,6,6-Trimethyl-1-cyclohexen-1-yl)-bicyclo[4.2.0]octa-1,3,5-triene
4-ethenyl-2,6-dimethoxyphenol
1-methyl-3-propylimidazolium iodide
4-acetamidobenzoic acid
morpholinium iodide
N,N-methylenebisacrylamide
4-methanesulfonyl-benzamidine hydrochloride
3-(2-aminoethyl)pyridine
magnesium oxide
poly(methyl methacrylate)
usnic acid
trimethylphenylammonium
formamidine acetate
zinc nitride
2,3-dimercaptopropanol
pyrrolidinium thiocyanate
thulium oxide
3,4,5-trifluoroaniline iodide
ethylenediamine
formamidinium
iron-doped zinc oxide
dodecylammonium formate
2,8-diiododibenzothiophene
methylammonium iodide
polyethylene oxide
copper(II)
cesium fluoride
methylammonium lead iodide
octylammonium
copper(I) oxide
methanol
phenylethylammonium iodide
sodium dodecyl sulfate
diethyl ether
phenethylammonium iodide
sodium chloride
sodium dodecyl sulfate
choline acetate
1-ethyl-3-methylimidazolium acetate
polyvinylidene fluoride
N,N′-dimethyl-1,3-propanediammonium dichloride
anethole
2,4-diamino-6-fluoropyri

KeyboardInterrupt: 

In [51]:
results

{'2-(6,6,6-Trimethyl-1-cyclohexen-1-yl)-bicyclo[4.2.0]octa-1,3,5-triene': None,
 '4-ethenyl-2,6-dimethoxyphenol': 'C(=C)C1=CC(=C(C(=C1)OC)O)OC',
 '1-methyl-3-propylimidazolium iodide': '[I-].CN1C=[N+](C=C1)CCC',
 '4-acetamidobenzoic acid': 'C(C)(=O)NC1=CC=C(C(=O)O)C=C1',
 'morpholinium iodide': '[I-].[NH2+]1CCOCC1',
 'N,N-methylenebisacrylamide': None,
 '4-methanesulfonyl-benzamidine hydrochloride': 'Cl.CS(=O)(=O)C1=CC=C(C(=N)N)C=C1',
 '3-(2-aminoethyl)pyridine': 'NCCC=1C=NC=CC1',
 'magnesium oxide': '[O-2].[Mg+2]',
 'poly(methyl methacrylate)': None,
 'usnic acid': None,
 'trimethylphenylammonium': 'C[N+](C1=CC=CC=C1)(C)C',
 'formamidine acetate': 'C(C)(=O)O.C(=N)N',
 'zinc nitride': None,
 '2,3-dimercaptopropanol': 'SC(CO)CS',
 'pyrrolidinium thiocyanate': '[S-]C#N.[NH2+]1CCCC1',
 'thulium oxide': '[O-2].[Tm+3].[O-2].[O-2].[Tm+3]',
 '3,4,5-trifluoroaniline iodide': '[I-].FC=1C=C(N)C=C(C1F)F',
 'ethylenediamine': 'C(CN)N',
 'formamidinium': 'C(=[NH2+])N',
 'iron-doped zinc oxide': Non